## This pipeline is for model training, evaluation and visualization

In [ ]:
import torch
from mamba_ssm import Mamba2
from mamba2_icl import Mamba2ICL   

In [ ]:
# Initialize model
model = Mamba2ICL(d_model=20).to("cuda")



In [ ]:
from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()




In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(10):
    for prompts in [train_linear, train_gaussian, train_dynamical]:
        for X, Y, x_query, y_query in prompts:
            optimizer.zero_grad()
            with autocast():
                # Concatenate context and query
                input_seq = torch.cat([torch.cat([X, Y.unsqueeze(-1)], dim=-1),
                                      torch.cat([x_query.unsqueeze(0), torch.zeros(1, 1)], dim=-1)], dim=0).to("cuda")
                output = model(input_seq.unsqueeze(0))[:, -1, :]  # Predict at query
                loss = torch.nn.functional.mse_loss(output, y_query.to("cuda"))
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()